In [171]:
import numpy as np
import pandas as pd
import RIfunctions as ri

In [172]:
ri_application_df = pd.read_csv('AR Scores.csv', sep = ',', index_col=0)
accept_df = pd.read_csv('input_file_2.csv', sep = ',', index_col=0)

C:\Users\joshy\AppData\Roaming\Python\Python38\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [173]:
# Filter for instances that were before 2016 as we know the accepted loans default status
# and should only infer those rejected applications for the same period

accept_df['issue_d'] = pd.to_datetime(accept_df['issue_d'])
ri_application_df['issue_d'] = pd.to_datetime(ri_application_df['issue_d'])

accept_df = accept_df.loc[accept_df['issue_d'] < '2016-1-1']
ri_application_df = ri_application_df.loc[ri_application_df['issue_d'] < '2016-1-1']

In [174]:
# Rows with Educational purpose have been dropped in the application dataset

accept_df = accept_df.drop(accept_df[accept_df['purpose'] == 'educational'].index)

In [175]:
print(accept_df.shape)
print(ri_application_df['rejected'].value_counts())

(646555, 21)
1    5544926
0     646555
Name: rejected, dtype: int64


In [176]:
print(accept_df.columns)
print(ri_application_df.columns)

Index(['loan_amnt', 'term', 'installment', 'emp_length', 'home_ownership',
       'verification_status', 'issue_d', 'purpose', 'dti', 'earliest_cr_line',
       'open_acc', 'pub_rec', 'revol_util', 'total_acc', 'application_type',
       'mort_acc', 'pub_rec_bankruptcies', 'log_annual_inc', 'fico_score',
       'log_revol_bal', 'charged_off'],
      dtype='object')
Index(['dti', 'emp_length', 'issue_d', 'loan_amnt', 'rejected',
       'purpose_Business', 'purpose_car', 'purpose_credit_card',
       'purpose_debt_consolidation', 'purpose_home_improvement',
       'purpose_house', 'purpose_major_purchase', 'purpose_medical',
       'purpose_moving', 'purpose_other', 'purpose_renewable_energy',
       'purpose_small_business', 'purpose_vacation', 'purpose_wedding',
       'score'],
      dtype='object')


In [177]:
rejects = ri_application_df.loc[ri_application_df["rejected"] ==1]

In [71]:
# # Simple assignment

# ri_df = pd.concat([accept_df, rejects], axis=0, ignore_index=True)
# ri_df.loc[ri_df['rejected'] == 1, 'charged_off'] = 1


In [72]:
# Random assignment

# neg, pos = np.bincount(accept_df['charged_off'])
# total = neg + pos
# reject_rate = pos / total

# print('Default Rate:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
#     total, pos, 100 * pos / total))

# random_assign_rate = reject_rate * 3.5

# print(reject_rate)
# print(random_assign_rate)

# # Higher the fraction, bigger the sample
# split_BAD = rejects.sample(frac = random_assign_rate, random_state=1)
# split_GOOD = rejects.drop(split_BAD.index)

# split_BAD['charged_off'] = 1
# split_GOOD = split_GOOD.assign(charged_off= 0)

# ri_df = pd.concat([accept_df, split_BAD, split_GOOD], axis=0, ignore_index=True)

In [ ]:
# # Assignment final cleaning

# ri_df.drop('rejected', axis=1, inplace=True)
# nan_cols = [i for i in ri_df.columns if ri_df[i].isna().any()]
# nan_cols = [e for e in nan_cols if e not in ('emp_length', 'purpose')]
# ri_df.drop(nan_cols, axis=1, inplace=True)
# ri_df = ri.purposeCleaning(ri_df)

In [178]:
# Augmentation

ri_application_df.sort_values(inplace=True, by="score", ascending = False)
accept_count = ri_application_df.loc[ri_application_df["rejected"] == 0, "rejected"].count()

aug_df = ri_application_df
aug_df['accept_score'] = 1 - aug_df['score']
intervals = np.array_split(aug_df, 10)

for band in intervals:
    accepts = band.loc[band["rejected"] == 0, "rejected"].count()
    rejects = band.loc[band["rejected"] == 1, "rejected"].count()
    weight = (rejects + accepts) / accepts
    band.drop(band[band['rejected'] == 1].index, inplace=True)
    band['weight'] = weight

aug_df=pd.concat(intervals)
aug_df.drop_duplicates(inplace=True)
ri_df = accept_df.merge(aug_df, on=["dti","emp_length","issue_d","loan_amnt","purpose"], how = 'inner')
print(ri_df.shape[0] - accept_df.shape[0])
ri_df.head()

In [187]:
ri_df.to_csv('RI_Accept.csv', mode='w+')